Nhiệm vụ 1 – Đọc & làm sạch dữ liệu: 
Mục tiêu : hiểu cấu trúc dữ liệu thô và xử lý các vấn đề ảnh hưởng đến phân tích. 
- Chuẩn hóa patient_id và full_name.
- Xử lý age âm hoặc dạng text.
- Chuẩn hóa gender về M/F.
- Làm sạch visit_date, department, diagnosis.
- Làm sạch total_fee và insurance_coverage.
- Xuất các file sạch để dùng cho các nhiệm vụ sau


 Đường dẫn và đọc dữ liệu

In [12]:
import pandas as pd
import os

# --- 1. CẤU HÌNH ĐƯỜNG DẪN (QUAN TRỌNG) ---
# Đường dẫn gốc đến thư mục dự án của bạn
base_path = r"D:\OneDrive\Máy tính\18A2_Nhom_4_Case_9"

# Đường dẫn Input (data_raw) và Output (data_clean)
input_path = os.path.join(base_path, "data_raw")
output_path = os.path.join(base_path, "data_clean")

print(f"Đọc dữ liệu từ: {input_path}")
print(f"Lưu dữ liệu về: {output_path}")

# --- 2. ĐỌC DỮ LIỆU ---
try:
    df_patient = pd.read_csv(os.path.join(input_path, "patient_info.csv"))
    df_visit = pd.read_csv(os.path.join(input_path, "visit_records.csv"))
    df_bill = pd.read_csv(os.path.join(input_path, "billing_info.csv"))
    print("-> Đã đọc 3 file CSV thành công!")
except FileNotFoundError:
    print("\nLỖI: Không tìm thấy file! Bạn hãy kiểm tra lại đường dẫn base_path.")
    raise

Đọc dữ liệu từ: D:\OneDrive\Máy tính\18A2_Nhom_4_Case_9\data_raw
Lưu dữ liệu về: D:\OneDrive\Máy tính\18A2_Nhom_4_Case_9\data_clean
-> Đã đọc 3 file CSV thành công!


- Xử lý age âm hoặc dạng text.
- Chuẩn hóa gender về M/F.

In [13]:
# --- 3. LÀM SẠCH PATIENT_INFO ---
# 3.1. Chuẩn hóa ID và Tên
df_patient['patient_id'] = df_patient['patient_id'].astype(str).str.strip().str.upper()
df_patient['full_name'] = df_patient['full_name'].astype(str).str.strip().str.title()
df_patient['full_name'] = df_patient['full_name'].str.replace(r'\s+', ' ', regex=True)

# 3.2. Xử lý Tuổi (Age)
# Trích xuất số (bỏ 't', 'tuoi'), chuyển sang số thực, lấy trị tuyệt đối (bỏ dấu âm)
df_patient['age'] = df_patient['age'].astype(str).str.extract(r'(\d+)', expand=False)
df_patient['age'] = pd.to_numeric(df_patient['age'], errors='coerce').abs()

# 3.3. Chuẩn hóa Giới tính (Gender)
gender_map = {'nam': 'M', 'm': 'M', 'male': 'M', 'nu': 'F', 'nữ': 'F', 'female': 'F', 'f': 'F'}
df_patient['gender'] = df_patient['gender'].astype(str).str.strip().str.lower().map(gender_map)


- Làm sạch visit_date, department, diagnosis.
- Làm sạch total_fee và insurance_coverage.

In [14]:
# --- 4. LÀM SẠCH VISIT_RECORDS ---
df_visit['visit_id'] = df_visit['visit_id'].astype(str).str.strip().str.upper()
df_visit['patient_id'] = df_visit['patient_id'].astype(str).str.strip().str.upper()

# 4.1. Làm sạch Ngày khám (visit_date)
# format='mixed' tự động xử lý cả dd/mm/yyyy và yyyy-mm-dd
df_visit['visit_date'] = pd.to_datetime(df_visit['visit_date'], dayfirst=True, format='mixed', errors='coerce')

# 4.2. Làm sạch Khoa (department)
df_visit['department'] = df_visit['department'].astype(str).str.strip().str.lower()
# Dùng Regex để sửa lỗi chính tả (VD: 'no i', 'noi' -> 'Nội')
dept_map_regex = {
    r'.*no\s*i.*': 'Nội',    # Bắt 'noi', 'no i'
    r'.*ngoai.*': 'Ngoại',
    r'.*san.*': 'Sản',
    r'.*nhi.*': 'Nhi'
}
df_visit['department'] = df_visit['department'].replace(dept_map_regex, regex=True).str.title()

# 4.3. Làm sạch Chẩn đoán (diagnosis)
# Thay ký tự đặc biệt (!@#$) và khoảng trắng bằng NaN, sau đó điền 'Chưa rõ'
df_visit['diagnosis'] = df_visit['diagnosis'].replace(r'^[\W_]+$', pd.NA, regex=True)
df_visit['diagnosis'] = df_visit['diagnosis'].fillna('Chưa rõ')

# --- 5. LÀM SẠCH BILLING_INFO ---
df_bill['visit_id'] = df_bill['visit_id'].astype(str).str.strip().str.upper()

# 5.1. Làm sạch Chi phí (total_fee)
# '200k' -> '200000', xóa 'đ', dấu chấm
df_bill['total_fee'] = df_bill['total_fee'].astype(str).str.lower().str.replace('k', '000', regex=False)
df_bill['total_fee'] = df_bill['total_fee'].str.replace(r'[^\d]', '', regex=True) # Chỉ giữ lại số
df_bill['total_fee'] = pd.to_numeric(df_bill['total_fee'], errors='coerce')

# 5.2. Làm sạch Bảo hiểm (insurance_coverage)
# Bỏ %, chuyển số âm thành dương
df_bill['insurance_coverage'] = df_bill['insurance_coverage'].astype(str).str.replace('%', '', regex=False)
df_bill['insurance_coverage'] = pd.to_numeric(df_bill['insurance_coverage'], errors='coerce').abs()
# Nếu > 1 (VD: 80) thì chia 100 để về thang đo 0-1 (VD: 0.8)
df_bill.loc[df_bill['insurance_coverage'] > 1, 'insurance_coverage'] = df_bill['insurance_coverage'] / 100


xuất file

In [15]:
# --- 6. XUẤT FILE ---
print("-" * 30)
print("Đang lưu file vào thư mục data_clean...")

# Kiểm tra xem folder output có tồn tại không trước khi lưu
if not os.path.exists(output_path):
    print(f"CẢNH BÁO: Thư mục '{output_path}' chưa tồn tại.")
    print("Bạn hãy tạo thư mục này thủ công rồi chạy lại code nhé!")
else:
    df_patient.to_csv(os.path.join(output_path, "patient_clean.csv"), index=False)
    df_visit.to_csv(os.path.join(output_path, "visit_clean.csv"), index=False)
    df_bill.to_csv(os.path.join(output_path, "billing_clean.csv"), index=False)
    
    print("HOÀN TẤT NHIỆM VỤ 1!")
    print("3 file sạch (patient_clean, visit_clean, billing_clean) đã được tạo.")

------------------------------
Đang lưu file vào thư mục data_clean...
HOÀN TẤT NHIỆM VỤ 1!
3 file sạch (patient_clean, visit_clean, billing_clean) đã được tạo.
